# BarrelNet Inference Notebook
This Notebook will run the code to run an inference pipeline for the trained Model, given a single Barrel point cloud sample as input. 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
from pathlib import Path

import dill as pickle
import numpy as np
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import trimesh
import plotly.graph_objects as go
import roma
import visu3d as v3d
import scipy.linalg

from barrelnet.pointnet.data import generate_cylinder_pts, prepare_point_cloud, normalize_pc, CylinderData
from mpl_toolkits.mplot3d import Axes3D
from torch.utils.data import Dataset, DataLoader
from barrelnet.pointnet.pointnet_utils import PointNetEncoder, feature_transform_reguliarzer
from barrelnet.pointnet.barrelnet import BarrelNet
from barrelnet.pointnet.data import CylinderDataOccluded, pts2inference_format
from barrelnet.pointnet.occlusion import render_occluded_point_cloud
from barrelnet.synthbarrel import random_cylinder_vol, random_cylinder_surf, monte_carlo_volume_ratio, generate_oriented_barrel, get_cyl_endpoints, get_cylinder_surf

In [ ]:
def get_trial_var(trialresults, varname):
    return [trial[varname] for trial in trialresults]

## Loading the occluded dataset (need to use Pointnet generation code)

In [ ]:
with open("data/synthbarrel/testbarrels_1000_fixed.pkl", "rb") as f:
    synthdict = pickle.load(f)
print(synthdict.keys())

In [ ]:
v3d.Point3d(p=synthdict["pts"][5].numpy()).fig

In [ ]:
## Load Model 
model_path = "checkpoints/pointnet_iter80_fixed.pth"
pointnet = BarrelNet(k=5, normal_channel=False)
pointnet.load_state_dict(torch.load(model_path))
pointnet.cuda().eval()

In [ ]:
# cylnp = random_cylinder_surf([0, 0, 0], [0, 0, height_ratio], 1, 5000).astype(np.float32)
# radius predicted: fraction of height
# normalized space: height is fixed at 1
# height_ratio = 2.5  # height / radius ratio
cylh = 1
ntrials = synthdict["radii"].shape[0]

trialresults = []
for i in tqdm(range(ntrials)):
# for i in tqdm(range()):
    results = {}
    cylnp = synthdict["pts"][i].numpy()
    axtruth = synthdict["axis_vectors"][i]
    rtruth = synthdict["radii"][i].numpy()
    # height in generated data is fixed at 1
    yoffsettruth = synthdict["burial_offsets"][i]
    x1truth, x2truth  = get_cyl_endpoints(axtruth, 1, yoffsettruth, axidx=1)
    
    results["axtruth"] = axtruth
    results["rtruth"] = rtruth
    results["yoffsettruth"] = yoffsettruth
    results["burialtruth"] = monte_carlo_volume_ratio(10000, x1truth, x2truth, rtruth, 0, 1, 0, 0)
    
    cylnp = cylnp.astype(np.float32)
    pts = torch.from_numpy(cylnp).cuda()
    pts, scale = pts2inference_format(pts)
    with torch.no_grad():
        radius_pred, yshift_pred, axis_pred = pointnet(pts)
        radius_pred = radius_pred.cpu().numpy()[0]
        yshift_pred = yshift_pred.cpu().numpy()[0]
        axis_pred = axis_pred.cpu().numpy()[0]
    axis_pred = axis_pred / np.linalg.norm(axis_pred)
    # scale predictions
    h = scale
    r = h * radius_pred
    y = yshift_pred
    x1pred, x2pred  = get_cyl_endpoints(axis_pred, h, y, axidx=1)
    
    results["axpred"] = axis_pred
    results["rpred"] = r
    results["yshiftpred"] = yshift_pred
    results["burialpred"] = monte_carlo_volume_ratio(10000, x1pred, x2pred, r, 0, 1, 0, 0)

    # print("ahAHSFHJKSADHJKFSDHJKDFSHJKFSAD")
    # print(axis_pred, r, h, y)
    # print(axtruth, rtruth, h, yoffsettruth / h)
    
    trialresults.append(results)

    # print("TRUTH")
    # print(f"axis: {cylax}\nradius: {cylr}\nheight: {cylh}\nz-offset: {cylz}")
    # print(f"burial percentage: {burialtruth}")
    # print("PREDICTED")
    # print(radius_pred, zshift_pred, axis_pred)
    # print(f"axis: {axis_pred}\nradius: {r}\nheight: {h}\nz-offset: {z}")
    # print(f"burial percentage: {burialpred}")

    # truthray = v3d.Ray(pos=[0,0,0], dir=cylax)
    # predray = v3d.Ray(pos=[0,0,0], dir=axis_pred)
    # v3d.make_fig([v3d.Point3d(p=cylnp), truthray, predray])
with open("results/pointnet_synth_results.pkl", "wb") as f:
    pickle.dump(trialresults, f)

In [ ]:
with open("results/pointnet_synth_results.pkl", "rb") as f:
    trialresults = pickle.load(f)

In [ ]:
i = 15255
trialres = trialresults[i]
x1truth, x2truth  = get_cyl_endpoints(trialres["axtruth"], 1, trialres["yoffsettruth"], axidx=1)
x1pred, x2pred  = get_cyl_endpoints(trialres["axpred"], 1, trialres["yshiftpred"], axidx=1)
randptsvol = random_cylinder_vol(x1truth, x2truth, trialres["rtruth"], 10000)
centroid_truth = (x1truth + x2truth) / 2
centroid_pred = (x1pred + x2pred) / 2
truthray = v3d.Ray(pos=centroid_truth, dir=trialres["axtruth"])
predray = v3d.Ray(pos=centroid_pred, dir=trialres["axpred"])
fig = v3d.make_fig([v3d.Point3d(p=synthdict["pts"][i].numpy()), truthray, predray])
x, y, z = get_cylinder_surf(x1truth, x2truth, trialres["rtruth"])
cylsurftruth = go.Surface(
    x=x, y=y, z=z,
    colorscale="purples",
    #  showscale=False,
    opacity=0.2)
fig.add_trace(cylsurftruth)
x, y, z = get_cylinder_surf(x1pred, x2pred, trialres["rpred"])
cylsurfpred = go.Surface(
    x=x, y=y, z=z,
    colorscale="oranges",
    #  showscale=False,
    opacity=0.2)
fig.add_trace(cylsurfpred)
fig.show()
print(monte_carlo_volume_ratio(10000, x1truth, x2truth, trialres["rtruth"], 0, 1, 0, 0))
print(monte_carlo_volume_ratio(10000, x1pred, x2pred, trialres["rpred"], 0, 1, 0, 0))

In [ ]:
cossims = np.abs(np.sum(np.array(get_trial_var(trialresults, "axtruth")) * np.array(get_trial_var(trialresults, "axpred")), axis=1))
plt.hist(cossims)
np.mean(cossims)

In [ ]:
np.argmin(cossims)

In [ ]:
np.sum(cossims < 0.9)

In [ ]:
burialerrs = np.abs(np.array(get_trial_var(trialresults, "burialtruth")) - np.array(get_trial_var(trialresults, "burialpred")))
plt.hist(burialerrs)
np.mean(burialerrs)

In [ ]:
yerrs = np.abs(np.array(get_trial_var(trialresults, "yoffsettruth")) - np.array(get_trial_var(trialresults, "yshiftpred")))
plt.hist(yerrs)
np.mean(yerrs)

In [ ]:
rerrs = np.abs(np.array(get_trial_var(trialresults, "rtruth")) - np.array(get_trial_var(trialresults, "rpred")))
plt.hist(rerrs)
np.mean(rerrs)

## test icp

In [ ]:
from sklearn.neighbors import KDTree

def icp_translate(source_pc, target_pc, max_iters=20, tol=1e-3, verbose=False, ntheta=3, nphi=3):
    """source_pc assumed to be smaller than target"""
    src_mean = np.mean(source_pc, axis=0)
    targ_mean = np.mean(target_pc, axis=0)
    scale = np.max(np.linalg.norm(target_pc - targ_mean, axis=1))
    src_cent = source_pc - src_mean
    targ_cent = target_pc - targ_mean

    src_kd = KDTree(source_pc)
    target_kd = KDTree(target_pc)
    thetas = np.linspace(0, 2 * np.pi, ntheta + 1)[:-1]
    phis = np.linspace(0, np.pi, nphi + 2)[1:-1]
    alltheta, allphi = np.meshgrid(thetas, phis)
    alltheta = alltheta.reshape(-1)
    allphi = allphi.reshape(-1)
    offset_choices = scale * np.array([np.sin(allphi) * np.cos(alltheta), np.sin(allphi) * np.sin(alltheta), np.cos(allphi)]).T
    alltranslations = np.zeros((len(alltheta), 3))
    allmeandists = np.zeros(len(alltheta))
    for j, offset in enumerate(offset_choices):
        # p = targ_mean - src_mean
        p = (targ_mean + offset) - src_mean
        prevp = p
        prevdist = np.inf
        K = max_iters
        for i in range(K):
            dists, close_idxs = target_kd.query(source_pc + p)
            meandist = np.mean(dists)
            targ_mean_filt = np.mean(target_pc[close_idxs], axis=0)
            p = targ_mean_filt - src_mean
            if np.abs(prevdist - meandist) < tol:
                if verbose:
                    print(f"converged at iter {i}")
                break
            prevp = p
            prevdist = meandist
            if i == K - 1:
                if verbose:
                    print(f"max iters {K} reached before tolerance {tol}")
        allmeandists[j] = np.mean(meandist)
        alltranslations[j, :] = p
    bestidx = np.argmin(allmeandists)
    pose = np.eye(4)
    pose[:3, 3] = alltranslations[bestidx]
    return pose

In [ ]:
i = 131
cylnp = synthdict["pts"][i].numpy().astype(np.float32)
np.random.seed(5)
cylnp[:, :] += np.random.uniform(-2, 2, 3)
cylpc = v3d.Point3d(p=cylnp)
trialres = trialresults[i]
x1truth, x2truth  = get_cyl_endpoints(trialres["axtruth"], 1, trialres["yoffsettruth"], axidx=1)
x1pred, x2pred  = get_cyl_endpoints(trialres["axpred"], 1, trialres["yshiftpred"], axidx=1)
x, y, z = get_cylinder_surf(x1pred, x2pred, trialres["rpred"])
origcylsurfpred = go.Surface(
    x=x, y=y, z=z,
    colorscale="purples",
    opacity=0.2)
randptssurfpred = random_cylinder_surf(x1truth, x2truth, trialres["rtruth"], 10000)
T = icp_translate(cylnp, randptssurfpred, max_iters=10, verbose=False, ntheta=3, nphi=3)
print(T)
x1pred -= T[:3, 3]
x2pred -= T[:3, 3]
centroid_pred = (x1pred + x2pred) / 2
truthray = v3d.Ray(pos=centroid_truth, dir=trialres["axtruth"])
predray = v3d.Ray(pos=centroid_pred, dir=trialres["axpred"])
fig = v3d.make_fig([cylpc, predray])
x, y, z = get_cylinder_surf(x1pred, x2pred, trialres["rpred"])
cylsurfpred = go.Surface(
    x=x, y=y, z=z,
    colorscale="oranges",
    opacity=0.2)
fig.add_trace(cylsurfpred)
fig.add_trace(origcylsurfpred)
fig.show()
print(monte_carlo_volume_ratio(10000, x1truth, x2truth, trialres["rtruth"], 0, 1, 0, 0))
print(monte_carlo_volume_ratio(10000, x1pred, x2pred, trialres["rpred"], 0, 1, 0, 0))

## non-occluded cylinder metrics (outdated probably)

In [ ]:
# cylnp = random_cylinder_surf([0, 0, 0], [0, 0, height_ratio], 1, 5000).astype(np.float32)
cylh = 1
cylr = cylh / height_ratio
ntrials = 5000

trialresults = []
for i in tqdm(range(ntrials)):
    results = {}
    cylnp, _, cylax, cylz = generate_oriented_barrel(cylr, cylh, 5000, sigma=0.05, zlims=[-0.3, 0.3])
    x1truth, x2truth = get_cyl_endpoints(cylax, height_ratio, cylz)
    burialtruth = monte_carlo_volume_ratio(10000, x1truth, x2truth, cylr, 0, 0, 1, 0)
    results["axtruth"] = cylax
    results["pc"] = cylnp
    results["ztruth"] = cylz
    results["rtruth"] = cylr
    results["x1truth"] = x1truth
    results["x2truth"] = x2truth
    results["burialtruth"] = burialtruth

    cylnp = cylnp.astype(np.float32)
    pts = torch.from_numpy(cylnp).cuda()
    pts, scale = pts2inference_format(pts)
    with torch.no_grad():
        radius_pred, zshift_pred, axis_pred = pointnet(pts)
        radius_pred = radius_pred.cpu().numpy()[0]
        zshift_pred = zshift_pred.cpu().numpy()[0]
        axis_pred = axis_pred.cpu().numpy()[0]
    axis_pred = axis_pred / np.linalg.norm(axis_pred)
    results["axpred"] = axis_pred
    # scale predictions
    r = radius_pred * scale
    h = r * height_ratio
    z = zshift_pred * h
    results["zpred"] = z
    results["rpred"] = r
    x1, x2 = get_cyl_endpoints(axis_pred, h, z)
    results["x1pred"] = x1
    results["x2pred"] = x2

    burialpred = monte_carlo_volume_ratio(10000, x1, x2, r, 0, 0, 1, 0)
    results["burialpred"] = burialpred
    results["cos_sim"] = np.abs(cylax @ axis_pred)
    results["burialerr"] = np.abs(burialtruth - burialpred)
    
    trialresults.append(results)

    # print("TRUTH")
    # print(f"axis: {cylax}\nradius: {cylr}\nheight: {cylh}\nz-offset: {cylz}")
    # print(f"burial percentage: {burialtruth}")
    # print("PREDICTED")
    # print(radius_pred, zshift_pred, axis_pred)
    # print(f"axis: {axis_pred}\nradius: {r}\nheight: {h}\nz-offset: {z}")
    # print(f"burial percentage: {burialpred}")

    # truthray = v3d.Ray(pos=[0,0,0], dir=cylax)
    # predray = v3d.Ray(pos=[0,0,0], dir=axis_pred)
    # v3d.make_fig([v3d.Point3d(p=cylnp), truthray, predray])

In [ ]:
cos_sims = get_trial_var(trialresults, "cos_sim")
burial_errs = get_trial_var(trialresults, "burialerr")
print(np.mean(cos_sims), np.std(cos_sims))
print(np.mean(burial_errs), np.std(burial_errs))

In [ ]:
plt.hist(cos_sims)

In [ ]:
np.sum(np.array(cos_sims) < 0.9)

In [ ]:
plt.hist(get_trial_var(trialresults, "burialtruth"))

In [ ]:
plt.hist(burial_errs)

In [ ]:
ztruths = get_trial_var(trialresults, "ztruth")
zpreds = get_trial_var(trialresults, "zpred")
zerrs = np.abs(np.array(ztruths) - zpreds)
plt.hist(zerrs)

In [ ]:
rtruths = get_trial_var(trialresults, "rtruth")
rpreds = get_trial_var(trialresults, "rpred")
rerrs = np.abs(np.array(rtruths) - rpreds)
plt.hist(rerrs)

In [ ]:
print(np.mean(zerrs), np.std(zerrs))
print(np.mean(rerrs), np.std(rerrs))

In [ ]:
axtruths = get_trial_var(trialresults, "axtruth")
axpreds = get_trial_var(trialresults, "axpred")
pcs = get_trial_var(trialresults, "pc")
worstidx = np.argmin(cos_sims)
# worstidx = 12
truthray = v3d.Ray(pos=[0,0,0], dir=axtruths[worstidx])
predray = v3d.Ray(pos=[0,0,0], dir=axpreds[worstidx])
v3d.make_fig([v3d.Point3d(p=pcs[worstidx]), truthray, predray])

In [ ]:
worstidx

In [ ]:
trialresults[worstidx]

In [ ]:
trialres = trialresults[3119]
x, y, z = get_cylinder_surf(trialres["x1pred"], trialres["x2pred"], trialres["rpred"])
centroid_truth = (trialres["x1truth"] - trialres["x2truth"]) / 2 + trialres["x2truth"]
centroid_pred = (trialres["x1pred"] - trialres["x2pred"]) / 2 + trialres["x2pred"]
truthray = v3d.Ray(pos=centroid_truth, dir=trialres["axtruth"])
predray = v3d.Ray(pos=centroid_pred, dir=trialres["axpred"])
fig = v3d.make_fig([v3d.Point3d(p=trialres["pc"][::5]), truthray, predray])
cyl1 = go.Surface(
    x=x, y=y, z=z,
    #  colorscale = colorscale,
    #  showscale=False,
    opacity=0.2)
fig.add_trace(cyl1)
# fig = go.Figure(data=[cyl1])
fig.show()